In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score, classification_report
from app.text_processor_mlb import TextProcessor
import pickle

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [31]:
df = pd.read_pickle('df_detractores_train_mlb-Copy1.pkl')
df

,SITE,CUST_ID,END_DATE,NPS,DETRACTION_REASON,COMMENTS,SURVEY_ID,COMENT_T,reviewer,review,otros
0,MLB,154663216,2020-05-31,-1,Demoras en la entrega,Cumprir o que o foi contratado!,25383838,¡Cumplir lo que fue contratado!,Fede W,Ambiguo,NaN
1,MLB,352010613,2022-02-16,-1,Otros motivos,Cumprir com o que está escrito no anucio No m...,53880866,¡Cumplir lo que se escribe en el anuncio en mi...,Fede W,Ambiguo,NaN
2,MLB,390987164,2020-03-14,-1,Otros motivos,Não recebi o meu produto ! Minha casa tem câme...,23135250,¡No recibí mi producto! Mi casa tiene cámaras ...,Fede W,Ambiguo,NaN
3,MLB,289003044,2020-02-11,-1,Otros motivos,A jad log disse que a minha mercadoria estava ...,21734725,Jad Log dijo que mi mercancía estaba defendien...,Fede W,Ambiguo,NaN
4,MLB,60871020,2023-01-04,-1,Otros motivos,"paguei,nao recebi o produto e nao recebi o din...",14924247-9877-4979-953b-294cea62f33c,"Pagué, no recibí el producto y no recibí el di...",Fede W,Ambiguo,NaN
...,...,...,...,...,...,...,...,...,...,...,...
14451,MLB,621832189,2020-11-27,-1,Precio del producto,preco melhor,31541382,Mejor precio,Uriel,Precio del producto,NaN
14452,MLB,354385061,2021-10-13,-1,Precio del producto,Paguei produto original mais veio replica,46291528,Pagué el producto original más Venir réplica,Uriel,No estoy conforme con el producto,NaN
14453,MLB,661025542,2021-10-19,-1,Precio del producto,"Pagar realmente o valor ,porque comprei um pro...",46792773,"Realmente pagar la cantidad, porque compré un ...",Uriel,Otros motivos,NaN
14454,MLB,132624520,2023-02-15,-1,Precio del producto,Concorrencia,11bc32cb-7e20-412f-9d13-eacc3cce2eb6,Competencia,Uriel,Outlier,NaN


In [12]:
df2 = pd.read_pickle('df_detractores_train-Copy1.pkl')
df2

,ORDER_ID,Site,NOTA_NPS,NPS,COMMENTS,END_DATE_MONTH,DETRACTION_REASON
0,2.521435e+09,MLC,4.0,-1.0,Lo publicitan como producto con stock inmediat...,202006.0,Demoras en la entrega
2,2.287362e+09,MLA,4.0,-1.0,Más información sobre las entregas y ser más e...,202001.0,Demoras en la entrega
3,4.824093e+09,MLM,6.0,-1.0,"Si la entrega es full entonces respetarlo, por...",202109.0,Demoras en la entrega
4,2.426763e+09,MLM,4.0,-1.0,Para empezar responder a la queja. Era un pedi...,202005.0,Otros
5,2.353216e+09,MLA,0.0,-1.0,Lo tenía que haber recibido el viernes y todav...,202003.0,Demoras en la entrega
...,...,...,...,...,...,...,...
11294,4.972552e+09,MLA,6.0,-1.0,estan los productos con mucho aumento,202111.0,Precio del producto
11295,NaN,NaN,NaN,NaN,aparece el mismo producto exacto del mismo ven...,NaN,Precio del producto
11296,NaN,NaN,NaN,NaN,los precios son diferentes a los de la tiendas...,NaN,Precio del producto
11297,NaN,NaN,NaN,NaN,todos los vendedores se quejan de las altas co...,NaN,Precio del producto


In [14]:
df2 = df2.loc[(df2['DETRACTION_REASON']!= 'Outlier') & (df2['DETRACTION_REASON']!= 'Otros'), :]
df2['categories'] = pd.Categorical(df2.DETRACTION_REASON)
df2['y'] = df2.categories.cat.codes
df2['COMMENTS_PREPROCESADOS'] = df2['COMMENTS'].apply(lambda x: TextProcessor().transform(pd.DataFrame([x])[0])[0])
df2

/tmp/ipykernel_4902/393304284.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['categories'] = pd.Categorical(df2.DETRACTION_REASON)
/tmp/ipykernel_4902/393304284.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['y'] = df2.categories.cat.codes
/tmp/ipykernel_4902/393304284.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

,ORDER_ID,Site,NOTA_NPS,NPS,COMMENTS,END_DATE_MONTH,DETRACTION_REASON,categories,y,COMMENTS_PREPROCESADOS
0,2.521435e+09,MLC,4.0,-1.0,Lo publicitan como producto con stock inmediat...,202006.0,Demoras en la entrega,Demoras en la entrega,3,lo publicitan product con stock inmediat indic...
2,2.287362e+09,MLA,4.0,-1.0,Más información sobre las entregas y ser más e...,202001.0,Demoras en la entrega,Demoras en la entrega,3,informacion sobr las entreg y especif con los ...
3,4.824093e+09,MLM,6.0,-1.0,"Si la entrega es full entonces respetarlo, por...",202109.0,Demoras en la entrega,Demoras en la entrega,3,si la entreg es full entonc respetarl mi play ...
5,2.353216e+09,MLA,0.0,-1.0,Lo tenía que haber recibido el viernes y todav...,202003.0,Demoras en la entrega,Demoras en la entrega,3,lo ten hab recib el viern y todav lo he recib ...
6,4.068852e+09,MLA,1.0,-1.0,Tener mas contacto con el cliente. Aun no lleg...,202010.0,Demoras en la entrega,Demoras en la entrega,3,ten contact con el client aun lleg una compr h...
...,...,...,...,...,...,...,...,...,...,...
11294,4.972552e+09,MLA,6.0,-1.0,estan los productos con mucho aumento,202111.0,Precio del producto,Precio del producto,6,estan los product con much aument
11295,NaN,NaN,NaN,NaN,aparece el mismo producto exacto del mismo ven...,NaN,Precio del producto,Precio del producto,6,aparec el mism product exact del mism vendedor...
11296,NaN,NaN,NaN,NaN,los precios son diferentes a los de la tiendas...,NaN,Precio del producto,Precio del producto,6,los preci son diferent los la tiend oficial y ...
11297,NaN,NaN,NaN,NaN,todos los vendedores se quejan de las altas co...,NaN,Precio del producto,Precio del producto,6,tod los vendedor quejan las altas comision y e...


In [32]:
df = df.loc[(df['review']!= 'Outlier') & (df['review']!= 'Otros motivos') & (df['review']!= 'Ambiguo') & (df['review']!= 'Problemas en el retiro en sucursal'), :]
df['categories'] = pd.Categorical(df.review)
df['y'] = df.categories.cat.codes
df['COMMENTS_PREPROCESADOS'] = df['COMMENTS'].apply(lambda x: TextProcessor().transform(pd.DataFrame([x])[0])[0])
df

/tmp/ipykernel_4902/3121882612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['categories'] = pd.Categorical(df.review)
/tmp/ipykernel_4902/3121882612.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['y'] = df.categories.cat.codes
/tmp/ipykernel_4902/3121882612.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

,SITE,CUST_ID,END_DATE,NPS,DETRACTION_REASON,COMMENTS,SURVEY_ID,COMENT_T,reviewer,review,otros,categories,y,COMMENTS_PREPROCESADOS
12,MLB,1772170,2022-10-14,-1,Otros motivos,Canal atendimento pós venda,a690edce-4678-4ff5-a8f9-112bbfc29d5f,Canal de servicio post -sales,Fede W,Atención al Cliente,NaN,Atención al Cliente,0,canal atend pos vend
13,MLB,264638193,2022-11-16,-1,No estoy conforme con el producto,Que nos dê uma melhor atenção pós venda!!,f61e53e5-690a-400b-847b-a4ce89f26512,¡Eso nos brinda una mejor atención después de ...,Fede W,Atención al Cliente,NaN,Atención al Cliente,0,melhor atenca pos vend
14,MLB,41295959,2020-10-03,-1,Otros motivos,serem mais rápidos em resposta ao problema do ...,29667954,Al ser más rápido en respuesta al problema del...,Fede W,Atención al Cliente,NaN,Atención al Cliente,0,ser rap respost problem client problem recent ...
15,MLB,770712880,2023-02-22,-1,Otros motivos,Melhorar a comunicação com o cliente,64778df2-1912-4efd-927c-54adf2123177,Mejorar la comunicación con el cliente,Fede W,Atención al Cliente,NaN,Atención al Cliente,0,melhor comunicaca client
16,MLB,1076024414,2023-03-26,-1,Devolución/Cancelación,"Bém comprei a bicicleta ergometrica, estou es...",c81f0b97-6807-4525-a976-29024dc555a7,"También compré la bicicleta ergométrica, esto...",Fede W,Cancelación/Devolución,NaN,Cancelación/Devolución,1,bem compr biciclet ergometr esper ima seman ag...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14447,MLB,29995387,2021-07-26,-1,Precio del producto,Um preço melhor seria a melhor opção.,41748520,Un mejor precio sería la mejor opción.,Uriel,Precio del producto,NaN,Precio del producto,6,prec melhor melhor opca
14449,MLB,128521036,2022-08-04,-1,Precio del producto,"Juros muito alto, dependendo da forma de pagam...",9f647cc0-7f92-4380-823a-b3a0067c6b8d,"Muy alto interés, dependiendo de la forma de pago",Uriel,Financiación,NaN,Financiación,4,jur alto depend form pagament
14450,MLB,506017932,2022-08-18,-1,Precio del producto,não recebi o produto,1939108f-cf5c-482c-9e3d-e9dafa7573f7,no recibí el producto,Uriel,Demoras en la entrega,NaN,Demoras en la entrega,3,nao receb produt
14451,MLB,621832189,2020-11-27,-1,Precio del producto,preco melhor,31541382,Mejor precio,Uriel,Precio del producto,NaN,Precio del producto,6,prec melhor


In [34]:
"""Train and test the model instance, from the given dataset."""
#df = df2.copy()
y_mapping = dict(enumerate(df['categories'].cat.categories))
target_names = list(y_mapping.values())

X_train_df, X_test_df, y_train, y_test = train_test_split(
                                                        df.drop(columns='y'),
                                                        df['y'].values,
                                                        test_size=0.3, 
                                                        random_state=123,
                                                        stratify=df['y'].values
)

X_train = X_train_df.COMMENTS_PREPROCESADOS.values
X_test = X_test_df.COMMENTS_PREPROCESADOS.values


tfidf_vectorizer_tri = TfidfVectorizer(ngram_range=(1, 3))
X_train_tfidf_tri = tfidf_vectorizer_tri.fit_transform(X_train)
X_test_tfidf_tri = tfidf_vectorizer_tri.transform(X_test)
X_train_tfidf_tri

<8040x86888 sparse matrix of type '<class 'numpy.float64'>'
	with 187771 stored elements in Compressed Sparse Row format>

In [27]:
X_train

array(['vcs inform nao faz entreg outr estad assim acontec comig der dias entreg pass temp avis tir mercador deposit betim min ger mor rio janeir',
       'agil not fiscal', 'qualidad ruim voc coloc empecilh devoluca',
       ..., 'ckmpar prec concorrent pois itens diferenc grand',
       'embalag extern danific embalag produt danific produt danific',
       'nao cob mot fic piqen faz'], dtype=object)

In [35]:
print(len(X_train))
print(len(y_train))

8040
8040


In [36]:
smote = SMOTE(random_state=123)
X_resampled, y_resampled = smote.fit_resample(X_train_tfidf_tri, y_train)
proba_model = SGDClassifier(loss='modified_huber')
proba_model.fit(X_resampled, y_resampled)
# metrics
y_pred = proba_model.predict(X_test_tfidf_tri)
metrics = {}
metrics['F1 general'] = f1_score(y_test, y_pred, average='weighted')
report = classification_report(y_test, y_pred, target_names=target_names, output_dict=True)

In [1]:
from pysentimiento import create_analyzer

In [2]:
sentimiento = create_analyzer(task="sentiment", lang="es")

In [3]:
sentimiento.predict('Estoy enojado')

AnalyzerOutput(output=NEG, probas={NEG: 0.878, NEU: 0.103, POS: 0.019})

In [2]:
import json
import base64
import pickle
import os

import pandas as pd
import numpy as np
from datetime import datetime

from melitk import logging
from melitk.fda2 import runtime
from melitk.melipass import get_secret
from melitk.bigquery import BigQueryClientBuilder

from app.data.batch_query_mlb import query_mlb

from melitk.fda2 import inventory
from getpass import getpass

os.environ["USER"]= 'tcampi'
tiger_token = getpass("Tiger token")
inventory.init(token=f"Bearer {tiger_token}")



Tiger token ································································································································································································································································································································································································································································································································································································································································································································································


2023-09-08T08:55:13-0400 - DEBUG: Initialized Lab context  [level: DEBUG]
2023-09-08T08:55:13-0400 - INFO: Token header: X-Tiger-Token  [level: INFO]
2023-09-08T08:55:13-0400 - INFO: Token header: X-Tiger-Token  [level: INFO]


In [20]:

class Predictor:
    def __init__(self, batch_data):
        self.batch_data = batch_data
    
    def predict_batch(self, comment, model):
        return model.predict(comment)
    
    def process_classifier_predictions(self, predictions):
        self.batch_data['preds'] = predictions
        new_cols = self.batch_data['preds'].apply(pd.Series)
        self.batch_data = pd.concat((self.batch_data, new_cols), axis=1)
        self.batch_data = self.batch_data.rename(columns={
                                                    'ORDER_ID': 'ORD_ORDER_ID',
                                                    'CUST_ID': 'CUS_CUST_ID',
                                                    'COMMENTS': 'COMMENT',
                                                    'END_DATE': 'RESPONSE_END_DT',
                                                    'predicted_clase1': 'DETRACTION_REASON_1',
                                                    'predicted_clase1_proba': 'DETRACTION_REASON_1_PROB',
                                                    'predicted_clase2': 'DETRACTION_REASON_2',
                                                    'predicted_clase2_proba': 'DETRACTION_REASON_2_PROB',
                                                    'NPS': 'NPS_2',
                                                    'NOTA_NPS': 'NPS'})
    
    def process_sentiment_predictions(self, predictions):
        self.batch_data['sent_preds'] = predictions
        self.batch_data['SENTIMENT'] = [self.batch_data.iloc[row]['sent_preds'].output for row in range(len(self.batch_data))]
        self.batch_data['SENTIMENT_PROB'] = [self.batch_data.iloc[row]['sent_preds'].probas[self.batch_data.iloc[row]['SENTIMENT']] for row in range(len(self.batch_data))]
        self.batch_data = self.batch_data.drop('sent_preds', axis=1)
        
    def process_emotion_predictions(self, predictions):
        self.batch_data['emotion_preds'] = predictions
        self.batch_data['NEGATIVE_EMOTION'] = [self.batch_data.iloc[row]['emotion_preds'].output for row in range(len(self.batch_data))]
        self.batch_data['NEGATIVE_EMOTION_PROB'] = [self.batch_data.iloc[row]['emotion_preds'].probas[self.batch_data.iloc[row]['NEGATIVE_EMOTION']] for row in range(len(self.batch_data))]
        self.batch_data.loc[self.batch_data['NEGATIVE_EMOTION'] == 'others', 'NEGATIVE_EMOTION_PROB'] = None
        self.batch_data.loc[self.batch_data['NEGATIVE_EMOTION'] == 'joy', 'NEGATIVE_EMOTION_PROB'] = None
        self.batch_data.loc[self.batch_data['NEGATIVE_EMOTION'] == 'others', 'NEGATIVE_EMOTION'] = None
        self.batch_data.loc[self.batch_data['NEGATIVE_EMOTION'] == 'joy', 'NEGATIVE_EMOTION'] = None
        self.batch_data = self.batch_data.drop('emotion_preds', axis=1)
        
    def process_columns(self):
        datetime_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        self.batch_data['AUD_FROM_INTERFACE'] = 'fury_cx-xm-nps-marketplace'
        self.batch_data['AUD_INS_DTTM'] = datetime_str
        self.batch_data['AUD_UPD_DTTM'] = datetime_str
        self.batch_data['AUD_TRANSACTION_ID'] = 0
        self.batch_data = self.batch_data[[
                                'ORD_ORDER_ID',
                                'CUS_CUST_ID',
                                'NPS',
                                'COMMENT',
                                'RESPONSE_END_DT',
                                'DETRACTION_REASON_1',
                                'DETRACTION_REASON_1_PROB',
                                'DETRACTION_REASON_2',
                                'DETRACTION_REASON_2_PROB',
                                'SENTIMENT',
                                'SENTIMENT_PROB',
                                'NEGATIVE_EMOTION',
                                'NEGATIVE_EMOTION_PROB',
                                'AUD_FROM_INTERFACE',
                                'AUD_INS_DTTM',
                                'AUD_UPD_DTTM',
                                'AUD_TRANSACTION_ID']]
        


In [9]:
query_mlb_hs = """WITH NPS AS(
            SELECT RES.*,
            CASE WHEN EXTRACT(MONTH FROM cast(RES.END_DATE as date)) BETWEEN 1 AND 3 THEN 'Q1'
                 WHEN EXTRACT(MONTH FROM cast(RES.END_DATE as date)) BETWEEN 4 AND 6 THEN 'Q2'
                 WHEN EXTRACT(MONTH FROM cast(RES.END_DATE as date)) BETWEEN 7 AND 9 THEN 'Q3'
                 WHEN EXTRACT(MONTH FROM cast(RES.END_DATE as date)) BETWEEN 10 AND 12 THEN 'Q4' END AS QUARTER,
            EXTRACT(YEAR FROM cast(RES.END_DATE as date)) AS YEAR
             FROM `meli-bi-data.SBOX_NPS_ANALYTICS.NPS_TX_MERGE` RES
            WHERE
                RES.ROL in ('B')
                -- AND RES.SHIPPING_SELLER in ('drop off','xd','fulfillment','ME Flex')
                and RES.SITE in ('MLB')
                and RES.NPS = -1
                and cast(RES.END_DATE as date) = current_date()-3
        )

        SELECT 
            SITE,
            BUYER_ID as CUST_ID,
            ORDER_ID,
            CAST(END_DATE as DATE) AS END_DATE,
            NOTA_NPS,
            NPS.NPS,
            DETRACTION_REASON_NPS AS DETRACTION_REASON,
            COMMENTS,
            SURVEY_ID
        FROM NPS"""

In [12]:
bigquery = BigQueryClientBuilder().with_encoded_secret('BQ_CREDENTIALS').build() #Esto para subir los datos a la tabla

#date = 'current_date()-3'
#dataset_query = query_mlb.format(date = date)
batch_data = bigquery.query_to_df(query_mlb_hs).df
batch_data = batch_data[batch_data['COMMENTS'].notna()]

batch_data

,SITE,CUST_ID,ORDER_ID,END_DATE,NOTA_NPS,NPS,DETRACTION_REASON,COMMENTS,SURVEY_ID
0,MLB,615730441.000000000,2000006336281560.000000000,2023-09-05,0E-9,-1,Custo do envio oferecido pelo vendedor,Comprei sem frete e agora está me cobrando fre...,7a2dc339-7bc3-43f3-bdca-668fc9d42928
1,MLB,70641272.000000000,2000006376601422.000000000,2023-09-05,0E-9,-1,A compra foi cancelada,"Entregar o que se compra, não entregar uma coi...",8400bc78-9d62-4732-a2b8-1849ed0cb1fd
2,MLB,790150689.000000000,2000006367462532.000000000,2023-09-05,2.000000000,-1,Não estou satisfeito com o produto,Melhorar a qualidade dos produtos,3b2a845a-d6ae-4306-87a3-266506951c0f
3,MLB,96194316.000000000,2000006361133408.000000000,2023-09-05,6.000000000,-1,Problemas com o envio a domicílio,"Segundo o vendedor, o problema com minha entre...",3908b01e-653c-4a0f-a22b-b759351ef900
5,MLB,1170949063.000000000,2000006356550468.000000000,2023-09-05,0E-9,-1,Não estou satisfeito com o produto,produto vencimento próximo,aecd9e50-6b4f-441f-a10b-9014031bd602
...,...,...,...,...,...,...,...,...,...
78,MLB,1067655945.000000000,2000006326174494.000000000,2023-09-05,2.000000000,-1,Problemas com o envio a domicílio,Data da entrega,dd7530f5-69e3-4e7e-af9d-4934b3fe0d3c
79,MLB,271110026.000000000,2000006340279242.000000000,2023-09-05,0E-9,-1,Outros motivos,NÃO RECEBI MINHA COMPRA ATÉ AGORA . COMO POSSO...,640ef8eb-5c1f-4f3d-8474-e8c5dd3c13de
80,MLB,219552705.000000000,2000006357677766.000000000,2023-09-05,0E-9,-1,Não estou satisfeito com o produto,"Melhorar a qualidade do produto, o que não con...",288358f8-6635-40a9-aeda-5960a27536d4
81,MLB,183356990.000000000,2000006330610252.000000000,2023-09-05,2.000000000,-1,Problemas com o envio a domicílio,"Vocês precisam melhorar o atendimento, ainda n...",2020fbc5-15d2-4883-b9b7-36db67718b86


In [15]:
#PARA HSP

sentiment_artifact = inventory.get(artifact_id='28c5136d-c6d8-437c-92e9-ce4efa22671e')
sentiment_model = pickle.loads(sentiment_artifact.load_to_bytes())

model_artifact = inventory.get(artifact_id='97a28ce9-f2c5-4cb0-89a5-ceaa22d8a1fc')
model = pickle.loads(model_artifact.load_to_bytes())

emotion_artifact = inventory.get(artifact_id='899ed399-2527-45c6-b0b4-2543ab1e8beb')
emotion_model = pickle.loads(emotion_artifact.load_to_bytes())


predictor = Predictor(batch_data.copy())
vfunc = np.vectorize(predictor.predict_batch)

# clasification model
preds = vfunc(predictor.batch_data.COMMENTS, model)
predictor.process_classifier_predictions(preds)

# sentiment model
preds = vfunc(predictor.batch_data.COMMENT, sentiment_model)
predictor.process_sentiment_predictions(preds)

# negative emotion model
preds = vfunc(predictor.batch_data.COMMENT, emotion_model)
predictor.process_emotion_predictions(preds)

# aud columns for bq and final preprocessing
predictor.process_columns()

2023-09-08T09:20:23-0400 - DEBUG: Getting artifact.  [level: DEBUG]
2023-09-08T09:20:24-0400 - INFO: Deserialize artifact instance id: 28c5136d-c6d8-437c-92e9-ce4efa22671e  [level: INFO]
2023-09-08T09:20:24-0400 - DEBUG: Getting url to download an artifact.  [level: DEBUG]
2023-09-08T09:20:25-0400 - DEBUG: Open signed URL.  [level: DEBUG]
2023-09-08T09:20:25-0400 - INFO: Compression? yes  [level: INFO]
2023-09-08T09:20:38-0400 - DEBUG: Getting artifact.  [level: DEBUG]
2023-09-08T09:20:38-0400 - INFO: Deserialize artifact instance id: 97a28ce9-f2c5-4cb0-89a5-ceaa22d8a1fc  [level: INFO]
2023-09-08T09:20:38-0400 - DEBUG: Getting url to download an artifact.  [level: DEBUG]
2023-09-08T09:20:38-0400 - DEBUG: Open signed URL.  [level: DEBUG]
2023-09-08T09:20:38-0400 - INFO: Compression? yes  [level: INFO]
2023-09-08T09:20:39-0400 - DEBUG: Getting artifact.  [level: DEBUG]
2023-09-08T09:20:39-0400 - INFO: Deserialize artifact instance id: 899ed399-2527-45c6-b0b4-2543ab1e8beb  [level: INFO]
2

KeyboardInterrupt: 

In [19]:
batch_data

,SITE,CUST_ID,ORDER_ID,END_DATE,NOTA_NPS,NPS,DETRACTION_REASON,COMMENTS,SURVEY_ID
0,MLB,615730441.000000000,2000006336281560.000000000,2023-09-05,0E-9,-1,Custo do envio oferecido pelo vendedor,Comprei sem frete e agora está me cobrando fre...,7a2dc339-7bc3-43f3-bdca-668fc9d42928
1,MLB,70641272.000000000,2000006376601422.000000000,2023-09-05,0E-9,-1,A compra foi cancelada,"Entregar o que se compra, não entregar uma coi...",8400bc78-9d62-4732-a2b8-1849ed0cb1fd
2,MLB,790150689.000000000,2000006367462532.000000000,2023-09-05,2.000000000,-1,Não estou satisfeito com o produto,Melhorar a qualidade dos produtos,3b2a845a-d6ae-4306-87a3-266506951c0f
3,MLB,96194316.000000000,2000006361133408.000000000,2023-09-05,6.000000000,-1,Problemas com o envio a domicílio,"Segundo o vendedor, o problema com minha entre...",3908b01e-653c-4a0f-a22b-b759351ef900
5,MLB,1170949063.000000000,2000006356550468.000000000,2023-09-05,0E-9,-1,Não estou satisfeito com o produto,produto vencimento próximo,aecd9e50-6b4f-441f-a10b-9014031bd602
...,...,...,...,...,...,...,...,...,...
78,MLB,1067655945.000000000,2000006326174494.000000000,2023-09-05,2.000000000,-1,Problemas com o envio a domicílio,Data da entrega,dd7530f5-69e3-4e7e-af9d-4934b3fe0d3c
79,MLB,271110026.000000000,2000006340279242.000000000,2023-09-05,0E-9,-1,Outros motivos,NÃO RECEBI MINHA COMPRA ATÉ AGORA . COMO POSSO...,640ef8eb-5c1f-4f3d-8474-e8c5dd3c13de
80,MLB,219552705.000000000,2000006357677766.000000000,2023-09-05,0E-9,-1,Não estou satisfeito com o produto,"Melhorar a qualidade do produto, o que não con...",288358f8-6635-40a9-aeda-5960a27536d4
81,MLB,183356990.000000000,2000006330610252.000000000,2023-09-05,2.000000000,-1,Problemas com o envio a domicílio,"Vocês precisam melhorar o atendimento, ainda n...",2020fbc5-15d2-4883-b9b7-36db67718b86


In [21]:
#PARA MLB

sentiment_artifact = inventory.get(artifact_id='e72ac50c-cf30-4b06-aebc-a694bf884a63')
sentiment_model = pickle.loads(sentiment_artifact.load_to_bytes())

model_artifact = inventory.get(artifact_id='4caf9acc-03c7-4e6b-8055-5b24d9162f24')
model = pickle.loads(model_artifact.load_to_bytes())

emotion_artifact = inventory.get(artifact_id='7ac93a3f-f4fd-4f55-b865-8a1d8e299b25')
emotion_model = pickle.loads(emotion_artifact.load_to_bytes())


predictor = Predictor(batch_data.copy())
vfunc = np.vectorize(predictor.predict_batch)

# clasification model
preds = vfunc(predictor.batch_data.COMMENTS, model)
predictor.process_classifier_predictions(preds)

# sentiment model
preds = vfunc(predictor.batch_data.COMMENT, sentiment_model)
predictor.process_sentiment_predictions(preds)

# negative emotion model
preds = vfunc(predictor.batch_data.COMMENT, emotion_model)
predictor.process_emotion_predictions(preds)

# aud columns for bq and final preprocessing
predictor.process_columns()


#   bigquery.fast_load(fname=csv_file_path, table_name=table_name, field_delimiter="|")


2023-09-08T09:26:48-0400 - DEBUG: Getting artifact.  [level: DEBUG]
2023-09-08T09:26:48-0400 - INFO: Deserialize artifact instance id: e72ac50c-cf30-4b06-aebc-a694bf884a63  [level: INFO]
2023-09-08T09:26:48-0400 - DEBUG: Getting url to download an artifact.  [level: DEBUG]
2023-09-08T09:26:48-0400 - DEBUG: Open signed URL.  [level: DEBUG]
2023-09-08T09:26:48-0400 - INFO: Compression? yes  [level: INFO]
2023-09-08T09:26:57-0400 - DEBUG: Getting artifact.  [level: DEBUG]
2023-09-08T09:26:57-0400 - INFO: Deserialize artifact instance id: 4caf9acc-03c7-4e6b-8055-5b24d9162f24  [level: INFO]
2023-09-08T09:26:57-0400 - DEBUG: Getting url to download an artifact.  [level: DEBUG]
2023-09-08T09:26:57-0400 - DEBUG: Open signed URL.  [level: DEBUG]
2023-09-08T09:26:57-0400 - INFO: Compression? yes  [level: INFO]
2023-09-08T09:26:57-0400 - DEBUG: Getting artifact.  [level: DEBUG]
2023-09-08T09:26:58-0400 - INFO: Deserialize artifact instance id: 7ac93a3f-f4fd-4f55-b865-8a1d8e299b25  [level: INFO]
2

In [35]:
"""
from google.cloud import bigquery as bigqueryOG

schema = [
    bigqueryOG.SchemaField("ORD_ORDER_ID", "INTEGER"),
    bigqueryOG.SchemaField("CUS_CUST_ID", "INTEGER"),
    bigqueryOG.SchemaField("NPS", "INTEGER"),
    bigqueryOG.SchemaField("COMMENT", "STRING"),
    bigqueryOG.SchemaField("RESPONSE_END_DT", "DATE"),
    bigqueryOG.SchemaField("DETRACTION_REASON_1", "STRING"),
    bigqueryOG.SchemaField("DETRACTION_REASON_1_PROB", "FLOAT64"),
    bigqueryOG.SchemaField("DETRACTION_REASON_2", "STRING"),
    bigqueryOG.SchemaField("DETRACTION_REASON_2_PROB", "FLOAT64"),
    bigqueryOG.SchemaField("SENTIMENT", "STRING"),
    bigqueryOG.SchemaField("SENTIMENT_PROB", "FLOAT64"),
    bigqueryOG.SchemaField("NEGATIVE_EMOTION", "STRING"),
    bigqueryOG.SchemaField("NEGATIVE_EMOTION_PROB", "FLOAT64"),
    bigqueryOG.SchemaField("AUD_FROM_INTERFACE", "STRING"),
    bigqueryOG.SchemaField("AUD_INS_DTTM", "DATETIME"),
    bigqueryOG.SchemaField("AUD_UPD_DTTM", "DATETIME"),
    bigqueryOG.SchemaField("AUD_TRANSACTION_ID", "INTEGER"),
    ]

In [35]:

staging = """
    CREATE OR REPLACE TABLE meli-bi-data.SBOX_CX_BI_ADS_CORE.STG_NPS_PREDICTIONS (
      ORD_ORDER_ID INTEGER, 
      CUS_CUST_ID INTEGER, 
      NPS INTEGER, 
      COMMENT STRING,
      RESPONSE_END_DT DATE,
      DETRACTION_REASON_1 STRING,
      DETRACTION_REASON_1_PROB FLOAT64,
      DETRACTION_REASON_2 STRING, 
      DETRACTION_REASON_2_PROB FLOAT64,
      SENTIMENT STRING,
      SENTIMENT_PROB FLOAT64,
      NEGATIVE_EMOTION STRING,
      NEGATIVE_EMOTION_PROB FLOAT64,
      AUD_FROM_INTERFACE STRING,
      AUD_INS_DTTM DATETIME,
      AUD_UPD_DTTM DATETIME,
      AUD_TRANSACTION_ID INTEGER
    )
"""
bigquery.execute(staging)

In [31]:
new_df = predictor.batch_data.copy()
print(new_df.dtypes)
new_df.head(5)

ORD_ORDER_ID                 object
CUS_CUST_ID                  object
NPS                          object
COMMENT                      object
RESPONSE_END_DT              dbdate
DETRACTION_REASON_1          object
DETRACTION_REASON_1_PROB    float64
DETRACTION_REASON_2          object
DETRACTION_REASON_2_PROB     object
SENTIMENT                    object
SENTIMENT_PROB              float64
NEGATIVE_EMOTION             object
NEGATIVE_EMOTION_PROB       float64
AUD_FROM_INTERFACE           object
AUD_INS_DTTM                 object
AUD_UPD_DTTM                 object
AUD_TRANSACTION_ID            int64
dtype: object


,ORD_ORDER_ID,CUS_CUST_ID,NPS,COMMENT,RESPONSE_END_DT,DETRACTION_REASON_1,DETRACTION_REASON_1_PROB,DETRACTION_REASON_2,DETRACTION_REASON_2_PROB,SENTIMENT,SENTIMENT_PROB,NEGATIVE_EMOTION,NEGATIVE_EMOTION_PROB,AUD_FROM_INTERFACE,AUD_INS_DTTM,AUD_UPD_DTTM,AUD_TRANSACTION_ID
0,2000006336281560.000000000,615730441.000000000,0E-9,Comprei sem frete e agora está me cobrando fre...,2023-09-05,Costo del envío,0.526730,Cancelación/Devolución,0.37171,NEG,0.846828,anger,0.62802,fury_cx-xm-nps-marketplace,2023-09-08 09:27:14,2023-09-08 09:27:14,0
1,2000006376601422.000000000,70641272.000000000,0E-9,"Entregar o que se compra, não entregar uma coi...",2023-09-05,Costo del envío,0.430102,Demoras en la entrega,0.462031,NEG,0.407179,None,NaN,fury_cx-xm-nps-marketplace,2023-09-08 09:27:14,2023-09-08 09:27:14,0
2,2000006367462532.000000000,790150689.000000000,2.000000000,Melhorar a qualidade dos produtos,2023-09-05,No estoy conforme con el producto,1.000000,Atención al Cliente,0.0,NEU,0.461465,None,NaN,fury_cx-xm-nps-marketplace,2023-09-08 09:27:14,2023-09-08 09:27:14,0
3,2000006361133408.000000000,96194316.000000000,6.000000000,"Segundo o vendedor, o problema com minha entre...",2023-09-05,Demoras en la entrega,0.536571,Problemas con el vendedor,0.362513,NEG,0.881119,None,NaN,fury_cx-xm-nps-marketplace,2023-09-08 09:27:14,2023-09-08 09:27:14,0
5,2000006356550468.000000000,1170949063.000000000,0E-9,produto vencimento próximo,2023-09-05,No estoy conforme con el producto,0.605443,Demoras en la entrega,0.187581,NEU,0.655210,None,NaN,fury_cx-xm-nps-marketplace,2023-09-08 09:27:14,2023-09-08 09:27:14,0


In [32]:
new_df.ORD_ORDER_ID = new_df.ORD_ORDER_ID.astype(int)
new_df.CUS_CUST_ID = new_df.CUS_CUST_ID.astype(int)
new_df.NPS = new_df.NPS.astype(int)
new_df.ORD_ORDER_ID = new_df.ORD_ORDER_ID.astype(int)
dataset2=new_df.loc[new_df['DETRACTION_REASON_1_PROB']!='',:].reset_index().drop(columns='index')
dataset3= dataset2.loc[dataset2['DETRACTION_REASON_2_PROB']!='',:].reset_index().drop(columns='index')
dataset3.loc[dataset3['NEGATIVE_EMOTION'].isnull(),['NEGATIVE_EMOTION']] = 'No emotion'
dataset3.loc[dataset3['NEGATIVE_EMOTION_PROB'].isnull(),['NEGATIVE_EMOTION_PROB']] = 1
dataset3.DETRACTION_REASON_2_PROB = dataset3.DETRACTION_REASON_2_PROB.astype(float)

In [34]:
print(dataset3.dtypes)


ORD_ORDER_ID                  int64
CUS_CUST_ID                   int64
NPS                           int64
COMMENT                      object
RESPONSE_END_DT              dbdate
DETRACTION_REASON_1          object
DETRACTION_REASON_1_PROB    float64
DETRACTION_REASON_2          object
DETRACTION_REASON_2_PROB    float64
SENTIMENT                    object
SENTIMENT_PROB              float64
NEGATIVE_EMOTION             object
NEGATIVE_EMOTION_PROB       float64
AUD_FROM_INTERFACE           object
AUD_INS_DTTM                 object
AUD_UPD_DTTM                 object
AUD_TRANSACTION_ID            int64
dtype: object


In [8]:
new_df.ORD_ORDER_ID = new_df.ORD_ORDER_ID.astype(int)
new_df.CUS_CUST_ID = new_df.CUS_CUST_ID.astype(int)
new_df.NPS = new_df.NPS.astype(int)
new_df.ORD_ORDER_ID = new_df.ORD_ORDER_ID.astype(int)
print(new_df.dtypes)
new_df.head()

ORD_ORDER_ID                  int64
CUS_CUST_ID                   int64
NPS                           int64
COMMENT                      object
RESPONSE_END_DT              dbdate
DETRACTION_REASON_1          object
DETRACTION_REASON_1_PROB    float64
DETRACTION_REASON_2          object
DETRACTION_REASON_2_PROB     object
SENTIMENT                    object
SENTIMENT_PROB              float64
NEGATIVE_EMOTION             object
NEGATIVE_EMOTION_PROB       float64
AUD_FROM_INTERFACE           object
AUD_INS_DTTM                 object
AUD_UPD_DTTM                 object
AUD_TRANSACTION_ID            int64
dtype: object


,ORD_ORDER_ID,CUS_CUST_ID,NPS,COMMENT,RESPONSE_END_DT,DETRACTION_REASON_1,DETRACTION_REASON_1_PROB,DETRACTION_REASON_2,DETRACTION_REASON_2_PROB,SENTIMENT,SENTIMENT_PROB,NEGATIVE_EMOTION,NEGATIVE_EMOTION_PROB,AUD_FROM_INTERFACE,AUD_INS_DTTM,AUD_UPD_DTTM,AUD_TRANSACTION_ID
0,2000004863339570,151323725,0,Compré dos productos con envío gratis. Hubo pr...,2023-01-14,Costo del envío,0.352341,Demoras en la entrega,0.188632,NEG,0.907590,anger,0.747406,fury_cx-xm-nps-marketplace,2023-08-18 09:16:49,2023-08-18 09:16:49,0
1,2000004905074250,622285380,5,Que llegué a tiempo las compras hace 2 semana...,2023-01-23,Demoras en la entrega,0.820367,Cancelación/Devolución,0.133851,NEG,0.565556,None,NaN,fury_cx-xm-nps-marketplace,2023-08-18 09:16:49,2023-08-18 09:16:49,0
2,2000004854986988,433849126,0,Escoger con cuidado los puntos de entrega porq...,2023-01-10,Problemas en el retiro en sucursal,0.400148,Demoras en la entrega,0.474963,NEG,0.906862,anger,0.545592,fury_cx-xm-nps-marketplace,2023-08-18 09:16:49,2023-08-18 09:16:49,0
3,2000004833899878,240613519,0,El vendedor no envío el producto y no hizo la ...,2023-01-08,Problemas con el vendedor,0.566080,Cancelación/Devolución,0.43392,NEG,0.685755,None,NaN,fury_cx-xm-nps-marketplace,2023-08-18 09:16:49,2023-08-18 09:16:49,0
4,2000004809004144,1108827874,5,Mas responsables en la entrega,2023-01-03,Demoras en la entrega,0.406361,Problemas con el vendedor,0.312569,NEU,0.609276,None,NaN,fury_cx-xm-nps-marketplace,2023-08-18 09:16:49,2023-08-18 09:16:49,0


In [23]:
new_df2=new_df.loc[new_df['DETRACTION_REASON_1_PROB']!='',:].reset_index().drop(columns='index')
new_df3= new_df2.loc[new_df2['DETRACTION_REASON_2_PROB']!='',:].reset_index().drop(columns='index')

In [27]:
new_df3.isnull().sum()

ORD_ORDER_ID                0
CUS_CUST_ID                 0
NPS                         0
COMMENT                     0
RESPONSE_END_DT             0
DETRACTION_REASON_1         0
DETRACTION_REASON_1_PROB    0
DETRACTION_REASON_2         0
DETRACTION_REASON_2_PROB    0
SENTIMENT                   0
SENTIMENT_PROB              0
NEGATIVE_EMOTION            0
NEGATIVE_EMOTION_PROB       0
AUD_FROM_INTERFACE          0
AUD_INS_DTTM                0
AUD_UPD_DTTM                0
AUD_TRANSACTION_ID          0
dtype: int64

In [25]:
new_df3.loc[new_df3['NEGATIVE_EMOTION'].isnull(),['NEGATIVE_EMOTION']] = 'No emotion'

In [26]:
new_df3.loc[new_df3['NEGATIVE_EMOTION_PROB'].isnull(),['NEGATIVE_EMOTION_PROB']] = 1

In [30]:
print(new_df3.dtypes)


ORD_ORDER_ID                 object
CUS_CUST_ID                  object
NPS                          object
COMMENT                      object
RESPONSE_END_DT              dbdate
DETRACTION_REASON_1          object
DETRACTION_REASON_1_PROB    float64
DETRACTION_REASON_2          object
DETRACTION_REASON_2_PROB    float64
SENTIMENT                    object
SENTIMENT_PROB              float64
NEGATIVE_EMOTION             object
NEGATIVE_EMOTION_PROB       float64
AUD_FROM_INTERFACE           object
AUD_INS_DTTM                 object
AUD_UPD_DTTM                 object
AUD_TRANSACTION_ID            int64
dtype: object


In [21]:
print(new_df3.dtypes)
table_name = "meli-bi-data.SBOX_CX_BI_ADS_CORE.STG_NPS_PREDICTIONS"
bigquery.df_to_gbq(dataset3, table_name, mode='replace')

ORD_ORDER_ID                  int64
CUS_CUST_ID                   int64
NPS                           int64
COMMENT                      object
RESPONSE_END_DT              dbdate
DETRACTION_REASON_1          object
DETRACTION_REASON_1_PROB    float64
DETRACTION_REASON_2          object
DETRACTION_REASON_2_PROB    float64
SENTIMENT                    object
SENTIMENT_PROB              float64
NEGATIVE_EMOTION             object
NEGATIVE_EMOTION_PROB       float64
AUD_FROM_INTERFACE           object
AUD_INS_DTTM                 object
AUD_UPD_DTTM                 object
AUD_TRANSACTION_ID            int64
dtype: object


2023-08-18T09:24:10-0400 - DEBUG: Retrying due to , sleeping 1.5s ...  [level: DEBUG]
2023-08-18T09:24:11-0400 - DEBUG: Retrying due to , sleeping 3.0s ...  [level: DEBUG]


In [29]:
new_df3.DETRACTION_REASON_2_PROB = new_df3.DETRACTION_REASON_2_PROB.astype(float)


In [22]:
new_df3.DETRACTION_REASON_1.value_counts()

Problemas con el vendedor             5123
Demoras en la entrega                 4921
No estoy conforme con el producto     4869
Cancelación/Devolución                1288
Costo del envío                        935
Precio del producto                    413
Atención al Cliente                    370
Problemas en el retiro en sucursal     235
Problemas con el pago                  112
Financiación                            18
Name: DETRACTION_REASON_1, dtype: int64

In [36]:
insert = """
    INSERT INTO meli-bi-data.SBOX_CX_BI_ADS_CORE.LK_CX_NPS_TX_PREDICTIONS(
      ORD_ORDER_ID, 
      CUS_CUST_ID, 
      NPS, 
      COMMENT,
      RESPONSE_END_DT,
      DETRACTION_REASON_1,
      DETRACTION_REASON_1_PROB,
      DETRACTION_REASON_2, 
      DETRACTION_REASON_2_PROB,
      SENTIMENT,
      SENTIMENT_PROB,
      NEGATIVE_EMOTION,
      NEGATIVE_EMOTION_PROB,
      AUD_FROM_INTERFACE,
      AUD_INS_DTTM,
      AUD_UPD_DTTM,
      AUD_TRANSACTION_ID    
    )
    SELECT
      ORD_ORDER_ID, 
      CUS_CUST_ID, 
      NPS, 
      COMMENT,
      RESPONSE_END_DT,
      DETRACTION_REASON_1,
      DETRACTION_REASON_1_PROB,
      DETRACTION_REASON_2, 
      DETRACTION_REASON_2_PROB,
      SENTIMENT,
      SENTIMENT_PROB,
      NEGATIVE_EMOTION,
      NEGATIVE_EMOTION_PROB,
      AUD_FROM_INTERFACE,
      CAST(AUD_INS_DTTM AS DATETIME),
      CAST(AUD_UPD_DTTM AS DATETIME),
      AUD_TRANSACTION_ID 
    FROM meli-bi-data.SBOX_CX_BI_ADS_CORE.STG_NPS_PREDICTIONS;
"""
bigquery.execute(insert)

In [ ]:
import json
import base64
import pickle
import os

import pandas as pd
import numpy as np
from datetime import datetime

from melitk import logging
from melitk.fda2 import runtime
from melitk.melipass import get_secret
from melitk.bigquery import BigQueryClientBuilder

from app.data.batch_query import query




logger = logging.getLogger(__name__)




def predict_batch(comment, model):
    return model.predict(comment)

def process_classifier_predictions(predictions):
    batch_data['preds'] = predictions
    new_cols = batch_data['preds'].apply(pd.Series)
    batch_data = pd.concat([batch_data, new_cols], axis=1)
    batch_data = batch_data.rename(columns={
                                                'ORDER_ID': 'ORD_ORDER_ID',
                                                'CUST_ID': 'CUS_CUST_ID',
                                                'COMMENTS': 'COMMENT',
                                                'END_DATE': 'RESPONSE_END_DT',
                                                'predicted_clase1': 'DETRACTION_REASON_1',
                                                'predicted_clase1_proba': 'DETRACTION_REASON_1_PROB',
                                                'predicted_clase2': 'DETRACTION_REASON_2',
                                                'predicted_clase2_proba': 'DETRACTION_REASON_2_PROB',
                                                'NPS': 'NPS_2',
                                                'NOTA_NPS': 'NPS'})

def process_sentiment_predictions(predictions):
    batch_data['sent_preds'] = predictions
    batch_data['SENTIMENT'] = [batch_data.iloc[row]['sent_preds'].output for row in range(len(batch_data))]
    batch_data['SENTIMENT_PROB'] = [batch_data.iloc[row]['sent_preds'].probas[batch_data.iloc[row]['SENTIMENT']] for row in range(len(batch_data))]
    batch_data = batch_data.drop('sent_preds', axis=1)

def process_emotion_predictions(predictions):
    batch_data['emotion_preds'] = predictions
    batch_data['NEGATIVE_EMOTION'] = [batch_data.iloc[row]['emotion_preds'].output for row in range(len(batch_data))]
    batch_data['NEGATIVE_EMOTION_PROB'] = [batch_data.iloc[row]['emotion_preds'].probas[batch_data.iloc[row]['NEGATIVE_EMOTION']] for row in range(len(batch_data))]
    batch_data.loc[batch_data['NEGATIVE_EMOTION'] == 'others', 'NEGATIVE_EMOTION_PROB'] = None
    batch_data.loc[batch_data['NEGATIVE_EMOTION'] == 'joy', 'NEGATIVE_EMOTION_PROB'] = None
    batch_data.loc[batch_data['NEGATIVE_EMOTION'] == 'others', 'NEGATIVE_EMOTION'] = None
    batch_data.loc[batch_data['NEGATIVE_EMOTION'] == 'joy', 'NEGATIVE_EMOTION'] = None
    batch_data = batch_data.drop('emotion_preds', axis=1)

def process_columns():
    datetime_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    batch_data['AUD_FROM_INTERFACE'] = 'fury_cx-xm-nps-marketplace'
    batch_data['AUD_INS_DTTM'] = datetime_str
    batch_data['AUD_UPD_DTTM'] = datetime_str
    batch_data['AUD_TRANSACTION_ID'] = 0
    batch_data = batch_data[[
                            'ORD_ORDER_ID',
                            'CUS_CUST_ID',
                            'NPS',
                            'COMMENT',
                            'RESPONSE_END_DT',
                            'DETRACTION_REASON_1',
                            'DETRACTION_REASON_1_PROB',
                            'DETRACTION_REASON_2',
                            'DETRACTION_REASON_2_PROB',
                            'SENTIMENT',
                            'SENTIMENT_PROB',
                            'NEGATIVE_EMOTION',
                            'NEGATIVE_EMOTION_PROB',
                            'AUD_FROM_INTERFACE',
                            'AUD_INS_DTTM',
                            'AUD_UPD_DTTM',
                            'AUD_TRANSACTION_ID']]
        

def load_to_bigquery(dataset, bigquery):
    
    dataset.ORD_ORDER_ID = dataset.ORD_ORDER_ID.astype(int)
    dataset.CUS_CUST_ID = dataset.CUS_CUST_ID.astype(int)
    dataset.NPS = dataset.NPS.astype(int)
    dataset.ORD_ORDER_ID = dataset.ORD_ORDER_ID.astype(int)
    dataset2=dataset.loc[dataset['DETRACTION_REASON_1_PROB']!='',:].reset_index().drop(columns='index')
    dataset3= dataset2.loc[dataset2['DETRACTION_REASON_2_PROB']!='',:].reset_index().drop(columns='index')
    dataset3.loc[dataset3['NEGATIVE_EMOTION'].isnull(),['NEGATIVE_EMOTION']] = 'No emotion'
    dataset3.loc[dataset3['NEGATIVE_EMOTION_PROB'].isnull(),['NEGATIVE_EMOTION_PROB']] = 1
    dataset3.DETRACTION_REASON_2_PROB = dataset3.DETRACTION_REASON_2_PROB.astype(float)

    table_name = "meli-bi-data.SBOX_CX_BI_ADS_CORE.LK_CX_NPS_TX_PREDICTIONS"
    
    staging = """
        CREATE OR REPLACE TABLE meli-bi-data.SBOX_CX_BI_ADS_CORE.STG_NPS_PREDICTIONS (
          ORD_ORDER_ID INTEGER, 
          CUS_CUST_ID INTEGER, 
          NPS INTEGER, 
          COMMENT STRING,
          RESPONSE_END_DT DATE,
          DETRACTION_REASON_1 STRING,
          DETRACTION_REASON_1_PROB FLOAT64,
          DETRACTION_REASON_2 STRING, 
          DETRACTION_REASON_2_PROB FLOAT64,
          SENTIMENT STRING,
          SENTIMENT_PROB FLOAT64,
          NEGATIVE_EMOTION STRING,
          NEGATIVE_EMOTION_PROB FLOAT64,
          AUD_FROM_INTERFACE STRING,
          AUD_INS_DTTM DATETIME,
          AUD_UPD_DTTM DATETIME,
          AUD_TRANSACTION_ID INTEGER
        )
    """
    bigquery.execute(staging)
    
    table_name_stg = "meli-bi-data.SBOX_CX_BI_ADS_CORE.STG_NPS_PREDICTIONS"
    bigquery.df_to_gbq(dataset, table_name_stg, mode='replace')
    
    insert = """
    INSERT INTO meli-bi-data.SBOX_CX_BI_ADS_CORE.LK_CX_NPS_TX_PREDICTIONS(
      ORD_ORDER_ID, 
      CUS_CUST_ID, 
      NPS, 
      COMMENT,
      RESPONSE_END_DT,
      DETRACTION_REASON_1,
      DETRACTION_REASON_1_PROB,
      DETRACTION_REASON_2, 
      DETRACTION_REASON_2_PROB,
      SENTIMENT,
      SENTIMENT_PROB,
      NEGATIVE_EMOTION,
      NEGATIVE_EMOTION_PROB,
      AUD_FROM_INTERFACE,
      AUD_INS_DTTM,
      AUD_UPD_DTTM,
      AUD_TRANSACTION_ID    
    )
    SELECT
      ORD_ORDER_ID, 
      CUS_CUST_ID, 
      NPS, 
      COMMENT,
      RESPONSE_END_DT,
      DETRACTION_REASON_1,
      DETRACTION_REASON_1_PROB,
      DETRACTION_REASON_2, 
      DETRACTION_REASON_2_PROB,
      SENTIMENT,
      SENTIMENT_PROB,
      NEGATIVE_EMOTION,
      NEGATIVE_EMOTION_PROB,
      AUD_FROM_INTERFACE,
      CAST(AUD_INS_DTTM AS DATETIME),
      CAST(AUD_UPD_DTTM AS DATETIME),
      AUD_TRANSACTION_ID 
    FROM meli-bi-data.SBOX_CX_BI_ADS_CORE.STG_NPS_PREDICTIONS;
    """
    bigquery.execute(insert)
    logger.info("New data inserted into LK_CX_NPS_TX_PREDICTIONS!")
    
    delete_table = """
    DROP TABLE meli-bi-data.SBOX_CX_BI_ADS_CORE.STG_NPS_PREDICTIONS;
    """
    bigquery.execute(delete_table)

        

bigquery = BigQueryClientBuilder().with_encoded_secret('BQ_CREDENTIALS').build() #Esto para subir los datos a la tabla

logger.info("Getting artifacts...")
batch_data = pickle.loads(runtime.inputs.artifacts["etl_dataset"].load_to_bytes()) #le cargo los artefactos para hacer las predicciones
model = pickle.loads(runtime.inputs.artifacts["classification_model"].load_to_bytes())
sentiment_model = pickle.loads(runtime.inputs.artifacts["sentiment_model"].load_to_bytes())
emotion_model = pickle.loads(runtime.inputs.artifacts["emotion_model"].load_to_bytes())

logger.info("Running Predictions...")
predictor = Predictor(batch_data.copy())
vfunc = np.vectorize(predictor.predict_batch)

# clasification model
preds = vfunc(predictor.batch_data.COMMENTS, model)
predictor.process_classifier_predictions(preds)

# sentiment model
preds = vfunc(predictor.batch_data.COMMENT, sentiment_model)
predictor.process_sentiment_predictions(preds)

# negative emotion model
preds = vfunc(predictor.batch_data.COMMENT, emotion_model)
predictor.process_emotion_predictions(preds)

# aud columns for bq and final preprocessing
predictor.process_columns()

logger.info("Uploading data to BigQuery...")
#    csv_file_path = os.path.abspath('batch_data.csv')
new_df = predictor.batch_data.copy()
load_to_bigquery(new_df, bigquery)
#   new_df.to_csv(csv_file_path, sep='|', index=False)
#   bigquery.fast_load(fname=csv_file_path, table_name=table_name, field_delimiter="|")
logger.info("Batch predictions done.")
